# Setup

In [1]:
import pandas as pd
import numpy as np
import os, time, stat, io, glob, pickle
from scipy.stats import gamma, norm
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('fivethirtyeight')
import plotly.io as pio

%load_ext autoreload
%autoreload 2
from covid_data_helper import *
from coronita_chart_helper import *
from coronita_model_helper import *
from coronita_web_helper import *
from coronita_bokeh_helper import *

from matplotlib.backends.backend_pdf import PdfPages


In [2]:
## DATA INGESTION ##

df_st_testing = get_covid19_tracking_data()

df_census = get_census_pop()

df_counties = get_complete_county_data()

counties_geo = get_counties_geo()

df_jhu_counties = get_jhu_counties()

df_st_testing_fmt = df_st_testing.copy()
df_st_testing_fmt = df_st_testing_fmt.rename(columns={'death':'deaths','positive':'cases'}).unstack('code')

df_interventions = get_state_policy_events()

df_goog_mob_us = get_goog_mvmt_us()
df_goog_mob_us = df_goog_mob_us[df_goog_mob_us.state.isnull()].set_index('dt')

df_goog_mob_state = get_goog_mvmt_state()

## MODEL PARAMETERS ##

covid_params = {}
covid_params['d_incub'] = 3.
covid_params['d_infect'] = 4.
covid_params['mort_rt'] = 0.01
covid_params['d_in_hosp'] = 11
covid_params['hosp_rt'] = 0.04
covid_params['d_to_hosp'] = 7.0
covid_params['d_in_hosp_mild'] = 11.0
covid_params['icu_rt'] = 13./41.
covid_params['d_in_icu'] = 13.0
covid_params['vent_rt'] = 0.4
covid_params['d_til_death'] =  17.0
covid_params['policy_trigger'] = True
covid_params['policy_trigger_once'] = True


State Testing Data Last Observation:  20200722


In [3]:
list_of_files = glob.glob('./output/df_fore_allstates_*.pkl') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)
df_fore_allstates = pd.read_pickle(latest_file)


./output/df_fore_allstates_20200723.pkl


In [4]:
list_of_files = glob.glob('./output/allstate_model_dicts_*.pkl') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)
with open(latest_file, 'rb') as handle:
    allstate_model_dicts = pickle.load(handle)

./output/allstate_model_dicts_20200723.pkl


# Development

In [43]:
from coronita_bokeh_helper import *
reset_output()
layout = bk_compare_exposures(df_census, df_fore_allstates)
output_notebook(); curdoc().theme = bk_theme
show(layout)

script_loc = "/assets/js/compare.js"
js, div = components(layout)
js = js[37:-9]
with io.open('../COVIDoutlook'+script_loc, mode='w', encoding='utf-8') as f:
    f.write(js)

script = '<script src="{}" async="True"></script>'.format(script_loc)
    
resources = CDN.render()

template = Template('''---
layout: page
title: Compare States
image: duotone2.png
---
{{ resources }}
{{ script }}
{{ div }}
''')

resources = CDN.render()

html = template.render(resources=resources,
                       script=script,
                       div=div)

with io.open('../COVIDoutlook/compare.md', mode='w', encoding='utf-8') as f:
    f.write(html)
    


Loading BokehJS ...

# COVIDoutlook

## US Overview

In [10]:
region_code = 'US'
model_dict = allstate_model_dicts[region_code]

df_agg = model_dict['df_agg']
scenario_name = 'No Change in Future Rᵗ Until 20% Hospital Capacity Trigger'
chart_title = "" #"{1} Scenario".format(model_dict['region_name'], scenario_name)
param_str = param_str_maker(model_dict)


from coronita_bokeh_helper import *
reset_output()
output_notebook()

p_cases = bk_positivetests(model_dict)
# p_cases.x_range = Range1d(pd.Timestamp('2020-03-01'), 
#                           model_dict['df_hist'].last_valid_index())
p_cases = bk_overview_layout(p_cases, 2)

p_tests = bk_totaltests(model_dict)
# p_tests.x_range = p_cases.x_range
p_tests = bk_overview_layout(p_tests, 2)

p_positivity = bk_postestshare(model_dict)
# p_positivity.x_range = p_cases.x_range
p_positivity = bk_overview_layout(p_positivity)

p_rt_conf = bk_rt_confid(model_dict, simplify=False)
# p_rt_conf.x_range = p_cases.x_range
p_rt_conf = bk_overview_layout(p_rt_conf)

p_googmvmt = bk_googmvmt(model_dict)
# p_googmvmt.x_range = p_cases.x_range
p_googmvmt = bk_overview_layout(p_googmvmt)

p_det_rt = bk_detection_rt(df_agg, model_dict)
# p_det_rt.x_range = p_cases.x_range
p_det_rt = bk_overview_layout(p_det_rt)

p_pop_share = bk_population_share(df_agg, model_dict, param_str, chart_title)
# p_pop_share.x_range = p_cases.x_range
p_pop_share = bk_overview_layout(p_pop_share)

curdoc().theme = bk_theme
r1 = [p_cases, p_tests, p_positivity, p_rt_conf,p_googmvmt,
      p_det_rt, p_pop_share]


script_loc = "/assets/js/us_overview.js"
js, div = components(r1)
js = js[37:-9]
with io.open('../COVIDoutlook'+script_loc, mode='w', encoding='utf-8') as f:
    f.write(js)

script = '<script src="{}" async="True"></script>'.format(script_loc)
    
resources = CDN.render()

template = Template('''---
layout: page
title: Home - US Overview
image: duotone-us.png
---
{{ resources }}
{{ script }}

<div class="w3-row-padding">
    <div class="w3-half">
        {{ div[0] }}
    </div>

    <div class="w3-half">
        {{ div[1] }}
    </div>
</div>
<hr>
{% for chart in div[2:-1] %}
    <div class="w3-container">
        {{ chart }}
    </div>
    <hr>
{% endfor %}
{{ exposure_prob }}
<hr>
{{ case_change }}
<hr>
<div class="w3-container">
    {{ div[-1] }}
</div>
''')

resources = CDN.render()
exposure_prob = '{% include_relative forecasts/plotly/ch_exposure_prob.html %}'
case_change = '{% include_relative forecasts/plotly/US_casepercap_cnty_map.html %}'

html = template.render(resources=resources,
                       script=script,
                       div=div,
                       exposure_prob=exposure_prob,
                       case_change=case_change
                      )

with io.open('../COVIDoutlook/index.html', mode='w', encoding='utf-8') as f:
    f.write(html)

Loading BokehJS ...

## Reproduction Rate Page

In [8]:
from coronita_bokeh_helper import *
reset_output()
output_notebook()
df_rts_allregs = pd.DataFrame()
df_wavg_rt_conf_allregs = pd.DataFrame()
l_rt_conf = []

model_dict = allstate_model_dicts['US']
l_rt_conf.append(bk_rt_confid(model_dict, True))
l_rt_conf[-1] = bk_overview_layout(l_rt_conf[-1], 1)

l_state_names = sorted([abbrev_us_state[code] for code in df_census.state.unique()])

for state_name in l_state_names:
    state_code = us_state_abbrev[state_name]
    model_dict = allstate_model_dicts[state_code]
    l_rt_conf.append(bk_rt_confid(model_dict, simplify=True))
    l_rt_conf[-1] = bk_repro_layout(l_rt_conf[-1], 2)


Loading BokehJS ...

In [9]:
curdoc().theme = bk_theme
script_loc = "/assets/js/rts.js"
js, div = components(l_rt_conf)
js = js[37:-9]
with io.open('../COVIDoutlook'+script_loc, mode='w', encoding='utf-8') as f:
    f.write(js)

script = '<script src="{}" async="True"></script>'.format(script_loc)
    
resources = CDN.render()

template = Template('''---
layout: page
title: Reproduction Rates
image: duotone4.png
---
{{ resources }}
{{ script }}

<div class="w3-row-padding">
    <div class="w3-container">
        {{ div[0] }}
    </div>
{% for chart in div[1:] %}
    <div class="w3-third">
        {{ chart }}
    </div>
{% endfor %}
</div>
''')

resources = CDN.render()

html = template.render(resources=resources,
                       script=script,
                       div=div)

with io.open('../COVIDoutlook/reproduction.md', mode='w', encoding='utf-8') as f:
    f.write(html)


In [10]:
for key in sorted(us_state_abbrev.keys()):
    print('<option value="/forecasts/{}">{}</option>'.format(us_state_abbrev[key], key))

<option value="/forecasts/AL">Alabama</option>
<option value="/forecasts/AK">Alaska</option>
<option value="/forecasts/AS">American Samoa</option>
<option value="/forecasts/AZ">Arizona</option>
<option value="/forecasts/AR">Arkansas</option>
<option value="/forecasts/CA">California</option>
<option value="/forecasts/CO">Colorado</option>
<option value="/forecasts/CT">Connecticut</option>
<option value="/forecasts/DE">Delaware</option>
<option value="/forecasts/DC">District of Columbia</option>
<option value="/forecasts/FL">Florida</option>
<option value="/forecasts/GA">Georgia</option>
<option value="/forecasts/GU">Guam</option>
<option value="/forecasts/HI">Hawaii</option>
<option value="/forecasts/ID">Idaho</option>
<option value="/forecasts/IL">Illinois</option>
<option value="/forecasts/IN">Indiana</option>
<option value="/forecasts/IA">Iowa</option>
<option value="/forecasts/KS">Kansas</option>
<option value="/forecasts/KY">Kentucky</option>
<option value="/forecasts/LA">Louisiana